<h1>Primo Progetto<h1>

In [2]:
# Importo le librerie richieste
import pandas as pd
from serpapi import GoogleScholarSearch
import json
import networkx as nx
import matplotlib.pyplot as plt
import os

In [3]:
GoogleScholarSearch.SERP_API_KEY = "0abb32444148ee2f5264785a1516d02307eb96aac922da2b728ebafd3683d690"

#### 1 | Utilizzo della libreria Python SerpAPI per scaricare, per ciascuno dei 7 autori elencati nel file nodes.csv

In [3]:
# Importo il file nodes.csv
df = pd.read_csv('data/nodes.csv')
display(df)

,name,affiliations
0,David La Barbera,Università Degli Studi Di Udine
1,Michael Soprano,Postdoctoral Research Fellow at the University...
2,Kevin Roitero,University of Udine
3,Stefano Mizzaro,Full professor of Computer Science and Informa...
4,Damiano Spina,"School of Computing Technologies, RMIT University"
5,Gianluca Demartini,Associate Professor at the University of Queen...
6,Eddy Maddalena,"Università degli Studi di Udine, Italy"


In [4]:

updated_df = pd.DataFrame(columns=['author_id', 'cited_by', 'interests'])

# Itero su ogni autore del dataframe
for index, row in df.iterrows():
    author_name = row['name']
    author_affliation = row['affiliations']
        
    params = {
        "engine" : "google_scholar_profiles",
        "hl": "en",
        "mauthors":f'{author_name},{author_affliation}'
    }
    search = GoogleScholarSearch(params)
    results = search.get_dict()['profiles']
    
    for profile in results:
        author_id = profile['author_id']
        cited_by = profile['cited_by']
        
        interests = []
        
        for interest in profile.get('interests', ):
            title = interest.get('title', '')
            interests.append(title)
    
    interests_str = ', '.join(interests)
    
    # Aggiungo una nuova riga al dataframe updated_df
    new_row = {'author_id': author_id, 'cited_by': cited_by, 'interests': interests_str}
    updated_df = pd.concat([updated_df, pd.DataFrame([new_row])], ignore_index=True)

new_df = pd.merge(df, updated_df, left_index=True, right_index=True)
display(new_df)



,name,affiliations,author_id,cited_by,interests
0,David La Barbera,Università Degli Studi Di Udine,MvN8wKEAAAAJ,139,Computer Science
1,Michael Soprano,Postdoctoral Research Fellow at the University...,ocK0qRUAAAAJ,210,"Crowdsourcing, Misinformation Assessment, Cogn..."
2,Kevin Roitero,University of Udine,1xd52jMAAAAJ,691,"Artificial Intelligence, Crowdsourcing and Hum..."
3,Stefano Mizzaro,Full professor of Computer Science and Informa...,2wvJC6IAAAAJ,4742,"Information retrieval, mobile systems, scholar..."
4,Damiano Spina,"School of Computing Technologies, RMIT University",sLzYrNYAAAAJ,2485,"Information Retrieval, Text Analytics, Data Sc..."
5,Gianluca Demartini,Associate Professor at the University of Queen...,PCAiILsAAAAJ,5352,"Information Retrieval, Semantic Web, Human Com..."
6,Eddy Maddalena,"Università degli Studi di Udine, Italy",egHDpFkAAAAJ,552,"Information Retrieval, Crowdsourcing, Data Sci..."


In [5]:
# Sovrascrivo il file nodes.csv con le nuove informazioni trovate
new_df.to_csv('data/nodes.csv', index=False)

#### 2a

In [6]:
# Carica il file CSV aggiornato con gli ID degli autori
df = pd.read_csv('data/nodes.csv')

# Crea un DataFrame vuoto per i coautori
coauthors_df = pd.DataFrame(columns=['name', 'affiliations', 'author_id', 'cited_by', 'interests'])

# Itera su ogni autore
for index, row in df.iterrows():
    author_id = row['author_id']
    
    # Recupera il profilo dell'autore utilizzando l'ID
    author_params = {
        "engine": "google_scholar_author",
        "author_id": author_id,
        "hl": "en"
    }
    author_search = GoogleScholarSearch(author_params)
    author_profile = author_search.get_dict()

    # Elenca i coautori
    coauthors = author_profile.get('co_authors', [])

    # Per ogni coautore, esegui una ricerca separata includendo le affiliazioni
    for coauthor in coauthors:
        coauthor_name = coauthor['name']
        coauthor_affiliation = coauthor.get('affiliations', '')  # Se disponibile

        # Ricerca il profilo del coautore
        coauthor_params = {
            "engine": "google_scholar_profiles",
            "hl": "en",
            "mauthors": f"{coauthor_name} {coauthor_affiliation}"
        }
        coauthor_search = GoogleScholarSearch(coauthor_params)
        coauthor_results = coauthor_search.get_dict()['profiles'][0]  # Prende il primo profilo come corretto

        # Crea una riga con le informazioni del coautore
        new_row = pd.DataFrame([{
            'name': coauthor_results['name'],
            'affiliations': coauthor_results.get('affiliations', ''),
            'author_id': coauthor_results.get('author_id', ''),
            'cited_by': coauthor_results.get('cited_by', ''),
            'interests': ', '.join([interest['title'] for interest in coauthor_results.get('interests', [])])
        }])

        # Aggiunge la nuova riga al DataFrame
        coauthors_df = pd.concat([coauthors_df, new_row], ignore_index=True)

# Rimuove i doppioni e ordina per nome
coauthors_df.drop_duplicates(inplace=True)
coauthors_df.sort_values(by='name', inplace=True)

# Salva il DataFrame in un file CSV
coauthors_df.to_csv('data/coauthors.csv', index=False)

#### 2b

In [ ]:
# Carica i DataFrame
df1 = pd.read_csv('data/nodes.csv')  # DataFrame degli autori originali
df2 = pd.read_csv('data/coauthors.csv')  # DataFrame dei coautori

# Concatena i DataFrame degli autori originali e dei coautori
combined_df = pd.concat([df1, df2], ignore_index=True)

# Ordina il DataFrame combinato per nome
combined_df.sort_values(by='name', inplace=True)

# Rimuovi eventuali duplicati basati sull'ID dell'autore (da provare anche senza "author_id")
combined_df.drop_duplicates(inplace=True) # combined_df.drop_duplicates(subset='author_id', inplace=True)

# Salva il DataFrame combinato in un file CSV
combined_df.to_csv('data/combined_authors.csv', index=False)

#### 2C

In [5]:
# Carica il file nodes.csv aggiornato con name, affiliations,author_id, cited_by e interests dei 7 autori originali
df = pd.read_csv('data/nodes.csv')

# Inizializza un DataFrame vuoto per le relazioni di coautorialità
edges_df = pd.DataFrame(columns=['author1', 'author2'])

# Itera su ogni autore originale
for index, row in df.iterrows():
    author_name = row['name']
    author_id = row['author_id']

    # Recupera il profilo dell'autore utilizzando l'ID
    author_params = {
        "engine": "google_scholar_author",
        "author_id": author_id,
        "hl": "en"
    }
    author_search = GoogleScholarSearch(author_params)
    author_profile = author_search.get_dict()

    # Elenca i coautori
    coauthors = author_profile.get('co_authors', [])

    # Per ogni coautore
    for coauthor in coauthors:
        coauthor_name = coauthor['name']
        # Crea un nuovo DataFrame con la relazione di coautorialità
        new_row = pd.DataFrame([{'author1': author_name, 'author2': coauthor_name}])
        # Aggiungi la relazione di coautorialità al DataFrame edges_df utilizzando pd.concat
        edges_df = pd.concat([edges_df, new_row], ignore_index=True)

# Salva il DataFrame in un file CSV
edges_df.to_csv('data/edges.csv', index=False)

In [ ]:
# Elimino nodes.csv e caouthors.csv

# Rinomino combined_authors.csv in nodes.csv

